In [1]:
import aiohttp
import lzma
import io

from datetime import datetime
from os.path import expanduser
from pathlib import Path

import pandas as pd

from rpki_analysis.delegated_stats import read_delegated_stats, StatsCombinedAllocations
from rpki_analysis.routinator import read_jsonext

In [2]:
df = await read_jsonext("https://rpki-validator.ripe.net/jsonext")

In [3]:
async with aiohttp.ClientSession() as session:
    async with session.get('https://ftp.ripe.net/pub/stats/ripencc/nro-stats/latest/nro-delegated-stats') as resp:
        df_delegated_stats = read_delegated_stats(io.StringIO(await resp.text()))
        
        lookup = StatsCombinedAllocations(df_delegated_stats[df_delegated_stats.status == "assigned"])

In [4]:
df_delegated_stats.status.unique()

['reserved', 'assigned', 'available']
Categories (3, object): ['assigned', 'available', 'reserved']

In [5]:
misses = list()

def maybe_lookup(row: pd.DataFrame) -> str:
    try:
        return lookup[row.prefix].rir
    except KeyError:
        misses.append(row)
        child_rirs = set(child.rir for child in lookup.children(row.prefix))
        return f"unknown-more-specific-allocs-{"-".join(child_rirs)}"
        
        return "unknown"

df['rir'] = df.apply(maybe_lookup, axis=1)
df_unmatched = pd.DataFrame.from_records(misses)

In [8]:
display(df_unmatched.prefix.unique())
display(df.keys())

array(['2c0f:ffd8::/32'], dtype=object)

Index(['type', 'uri', 'tal', 'not_before', 'not_after', 'chain_not_before',
       'chain_not_after', 'asn', 'prefix', 'max_length', 'rir'],
      dtype='object')

The code below detects some mis-alignment. Many of these will becaused by registration artifacts (two continous registrations controlled by one entity, that are under two IDs in the delegated stats.

In [8]:
df.loc[df.tal.str.replace("ripe", "ripencc") != df.rir, ['uri', 'tal', 'asn', 'prefix', 'max_length', 'rir']]

,uri,tal,asn,prefix,max_length,rir
19280,rsync://repo-rpki.idnic.net/repo/943e2e67-a171...,apnic,AS24532,27.111.32.0/19,24,unknown-more-specific-allocs-apnic
42355,rsync://rpki.cnnic.cn/rpki/A9162E3D0000/137/OX...,apnic,AS24426,43.236.0.0/16,16,unknown-more-specific-allocs-apnic
42356,rsync://rpki.cnnic.cn/rpki/A9162E3D0000/137/pg...,apnic,AS24426,43.239.0.0/19,19,unknown-more-specific-allocs-apnic
42357,rsync://rpki.cnnic.cn/rpki/A9162E3D0000/137/q0...,apnic,AS24426,43.239.32.0/20,20,unknown-more-specific-allocs-apnic
43001,rsync://rpki.cnnic.cn/rpki/A9162E3D0000/890/Y_...,apnic,AS63567,43.248.176.0/20,20,unknown-more-specific-allocs-apnic
120422,rsync://rpki.ripe.net/repository/DEFAULT/38/da...,ripe,AS200603,80.244.3.0/24,24,unknown-more-specific-allocs-
177467,rsync://rpki.apnic.net/member_repository/A918E...,apnic,AS45769,103.26.232.0/22,24,unknown-more-specific-allocs-apnic
177866,rsync://rpki.cnnic.cn/rpki/A9162E3D0000/137/pg...,apnic,AS24426,103.35.0.0/19,19,unknown-more-specific-allocs-apnic
177867,rsync://rpki.cnnic.cn/rpki/A9162E3D0000/137/q0...,apnic,AS24426,103.35.32.0/20,20,unknown-more-specific-allocs-apnic
188252,rsync://repo-rpki.idnic.net/repo/IDNIC-ID/2/AS...,apnic,AS150980,103.138.166.0/23,24,unknown-more-specific-allocs-apnic


In [9]:
home = Path(expanduser("~"))
now = datetime.now()

df[df.tal.str.replace("ripe", "ripencc") != df.rir].to_excel(home / f"Desktop/{now.strftime('%Y%m%d')}-results-results.xlsx")

In [10]:
df_unmatched

,type,uri,tal,not_before,not_after,chain_not_before,chain_not_after,asn,prefix,max_length
0,roa,rsync://rpki.afrinic.net/repository/member_rep...,afrinic,2023-07-06T17:56:40Z,2033-07-06T17:56:40Z,2024-01-01T03:48:39Z,2025-03-31T00:00:00Z,AS37105,2c0f:ffd8::/32,32
1,roa,rsync://rpki.afrinic.net/repository/member_rep...,afrinic,2023-07-06T17:56:40Z,2033-07-06T17:56:40Z,2024-01-01T03:48:39Z,2025-03-31T00:00:00Z,AS37105,2c0f:ffd8::/32,32
2,roa,rsync://rpki.afrinic.net/repository/member_rep...,afrinic,2023-07-06T17:56:40Z,2033-07-06T17:56:40Z,2024-01-01T03:48:39Z,2025-03-31T00:00:00Z,AS37105,2c0f:ffd8::/32,32
3,roa,rsync://rpki.afrinic.net/repository/member_rep...,afrinic,2023-07-06T17:56:40Z,2033-07-06T17:56:40Z,2024-01-01T03:48:39Z,2025-03-31T00:00:00Z,AS37105,2c0f:ffd8::/32,32
4,roa,rsync://rpki.afrinic.net/repository/member_rep...,afrinic,2023-07-06T17:56:40Z,2033-07-06T17:56:40Z,2024-01-01T03:48:39Z,2025-03-31T00:00:00Z,AS37105,2c0f:ffd8::/32,32
5,roa,rsync://rpki.afrinic.net/repository/member_rep...,afrinic,2023-07-06T17:56:40Z,2033-07-06T17:56:40Z,2024-01-01T03:48:39Z,2025-03-31T00:00:00Z,AS37105,2c0f:ffd8::/32,32
6,roa,rsync://rpki.afrinic.net/repository/member_rep...,afrinic,2023-07-06T17:56:40Z,2033-07-06T17:56:40Z,2024-01-01T03:48:39Z,2025-03-31T00:00:00Z,AS37105,2c0f:ffd8::/32,32
7,roa,rsync://rpki.afrinic.net/repository/member_rep...,afrinic,2023-07-06T17:56:40Z,2033-07-06T17:56:40Z,2024-01-01T03:48:39Z,2025-03-31T00:00:00Z,AS37105,2c0f:ffd8::/32,32
8,roa,rsync://rpki.afrinic.net/repository/member_rep...,afrinic,2023-07-06T17:56:40Z,2033-07-06T17:56:40Z,2024-01-01T03:48:39Z,2025-03-31T00:00:00Z,AS37105,2c0f:ffd8::/32,32
9,roa,rsync://rpki.afrinic.net/repository/member_rep...,afrinic,2023-07-06T17:56:40Z,2033-07-06T17:56:40Z,2024-01-01T03:48:39Z,2025-03-31T00:00:00Z,AS37105,2c0f:ffd8::/32,32


In [11]:
top_10 = df.groupby(["prefix", "asn", "rir", "max_length"]).count().nlargest(10, ['not_before'])
top_10

top_10.join(df.set_index(["prefix", "asn", "rir", "max_length"]), lsuffix="lhs")

typelhs  urilhs  tallhs  \
prefix           asn     rir    max_length                            
131.221.164.0/22 AS27901 lacnic 22                8       8       8   
                                22                8       8       8   
                                22                8       8       8   
                                22                8       8       8   
                                22                8       8       8   
...                                             ...     ...     ...   
2800:430::/32    AS23487 lacnic 48                7       7       7   
                                48                7       7       7   
                                48                7       7       7   
                                48                7       7       7   
                                48                7       7       7   

                                            not_beforelhs  not_afterlhs  \
prefix           asn     rir    max_length                                
131.221.164.0/22 AS27901 lacnic 22                      8             8   
                                22                      8             8   
                                22                      8             8   
                                22                      8             8   
                                22                      8             8   
...                                                   ...           ...   
2800:430::/32    AS23487 lacnic 48                      7             7   
                                48                      7             7   
                                48                      7             7   
                                48                      7             7   
                                48                      7             7   

                                            chain_not_beforelhs  \
prefix           asn     rir    max_length                        
131.221.164.0/22 AS27901 lacnic 22                            8   
                                22                            8   
                                22                            8   
                                22                            8   
                                22                            8   
...                                                         ...   
2800:430::/32    AS23487 lacnic 48                            7   
                                48                            7   
                                48                            7   
                                48                            7   
                                48                            7   

                                            chain_not_afterlhs type  \
prefix           asn     rir    max_length                            
131.221.164.0/22 AS27901 lacnic 22                           8  roa   
                                22                           8  roa   
                                22                           8  roa   
                                22                           8  roa   
                                22                           8  roa   
...                                                        ...  ...   
2800:430::/32    AS23487 lacnic 48                           7  roa   
                                48                           7  roa   
                                48                           7  roa   
                                48                           7  roa   
                                48                           7  roa   

                                                                                          uri  \
prefix           asn     rir    max_length                                                      
131.221.164.0/22 AS27901 lacnic 22          rsync://repository.lacnic.net/rpki/lacnic/7d48...   
                                22          rsync://repos

In [12]:
df.groupby(["uri", "asn"]).count()

,,type,tal,not_before,not_after,chain_not_before,chain_not_after,prefix,max_length,rir
uri,asn,,,,,,,,,
rsync://0.sb/repo/sb/1/3134362e31392e302e302f32342d3234203d3e2038383838.roa,AS8888,1,1,1,1,1,1,1,1,1
rsync://0.sb/repo/sb/1/3134362e31392e3134352e302f32342d3234203d3e2038383838.roa,AS8888,1,1,1,1,1,1,1,1,1
rsync://0.sb/repo/sb/1/326131313a333a3530303a3a2f34302d3438203d3e203433333537.roa,AS43357,1,1,1,1,1,1,1,1,1
rsync://0.sb/repo/sb/1/326131313a333a3a2f33322d3438203d3e2038383838.roa,AS8888,1,1,1,1,1,1,1,1,1
rsync://0.sb/repo/sb/1/326131313a343a3a2f33322d3438203d3e2038383838.roa,AS8888,1,1,1,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...
rsync://rsync.rpki.tianhai.link/repo/TianhaiRpki/0/326131333a313830303a3a2f34342d3438203d3e2034383432.roa,AS4842,1,1,1,1,1,1,1,1,1
rsync://rsync.rpki.tianhai.link/repo/TianhaiRpki/0/326131333a313830313a3138303a3a2f34332d3433203d3e2034383432.roa,AS4842,1,1,1,1,1,1,1,1,1
rsync://rsync.rpki.tianhai.link/repo/TianhaiRpki/0/326131333a313830313a3138303a3a2f34332d3438203d3e20313531343634.roa,AS151464,1,1,1,1,1,1,1,1,1


In [13]:
df.groupby(["uri", "asn"]).count().nlargest(10, ['prefix'])

,,type,tal,not_before,not_after,chain_not_before,chain_not_after,prefix,max_length,rir
uri,asn,,,,,,,,,
rsync://rpki.ripe.net/repository/DEFAULT/e2/88fdd1-8b7b-4101-ac66-b590957d9415/1/sqJb3m0bhHpyFZL7PATz3SmhUKQ.roa,AS39891,6483,6483,6483,6483,6483,6483,6483,6483,6483
rsync://rpki.apnic.net/member_repository/A91DFB70/2983647C838F11E586FC5812C4F9AE02/1862B318B01E11EEA92DFE61C4F9AE02.roa,AS9299,4263,4263,4263,4263,4263,4263,4263,4263,4263
rsync://rpki.ripe.net/repository/DEFAULT/34/41c321-347e-4c3e-affd-e2b527728e96/1/F8-dcpSK7eR4Q3eZ7DjNFD0hoS0.roa,AS8551,4127,4127,4127,4127,4127,4127,4127,4127,4127
rsync://rpki.ripe.net/repository/DEFAULT/2f/4b7ef8-8643-4dbb-92aa-0740115b5a52/1/HLyZ7EIByAmhVb-ajQtU8qwlGa4.roa,AS5416,3933,3933,3933,3933,3933,3933,3933,3933,3933
rsync://rpki.apnic.net/member_repository/A9197CED/83F56DAC1D8411E2910689DA08B02CD2/CEFF120A77B711EEACA2AA09C4F9AE02.roa,AS9829,3808,3808,3808,3808,3808,3808,3808,3808,3808
rsync://rpki.ripe.net/repository/DEFAULT/31/0762da-d66f-4dcb-9c7f-802bb51a1bed/1/UMs8uIBP7mybEF6K3AYBQntR1M8.roa,AS50710,2719,2719,2719,2719,2719,2719,2719,2719,2719
rsync://rpki.ripe.net/repository/DEFAULT/31/0762da-d66f-4dcb-9c7f-802bb51a1bed/1/UWNAJnUwK24DJTb0itLIyRvDuVo.roa,AS203214,2719,2719,2719,2719,2719,2719,2719,2719,2719
rsync://rpki.ripe.net/repository/DEFAULT/31/0762da-d66f-4dcb-9c7f-802bb51a1bed/1/r5tgyRrZAFq7SVMz8QPO9Cp27U0.roa,AS199739,2719,2719,2719,2719,2719,2719,2719,2719,2719
rsync://rpki.ripe.net/repository/DEFAULT/b0/742370-e10f-4541-aa6c-61afb66d1e8c/1/17Mmwz1MJTu5PiiELXXid20e0uk.roa,AS5384,2576,2576,2576,2576,2576,2576,2576,2576,2576


In [14]:
df.groupby(['publication_point']).count().nlargest(10, ['uri'])

KeyError: 'publication_point'

In [ ]:
df.groupby(['asn']).count().nlargest(10, ['prefix'])

In [ ]:
df_unique = df.reset_index().drop_duplicates(['uri', 'publication_point'])

In [ ]:
df_unique.groupby(['publication_point']).count().nlargest(10, ['prefix'])

In [ ]:
 df.reset_index().drop_duplicates(['uri', 'publication_point'])

In [ ]:
 df.reset_index().drop_duplicates(['publication_point'])

In [ ]:
# Now let's work on unique VRPs

In [ ]:
df = df.reset_index().drop_duplicates(['publication_point', 'asn', 'prefix', 'max_length'])

In [ ]:
df.groupby(["uri", "asn"]).count().nlargest(10, ['index'])

In [ ]:
df.drop_duplicates(['asn', 'publication_point']).groupby(['publication_point']).count().nlargest(10, ['prefix'])